# Exercice 1

In [21]:
import numpy as np
import pandas as pd
import datetime

In [1]:
import numpy as np
import pandas as pd
import datetime


# transactions à générer
n_transactions = 10000

# id uniques
ids = list(range(1, n_transactions + 1))

# product_id
product_ids = np.random.randint(1, 113, n_transactions)

# Attention ! les prix unitaires dépendent du produit_id
unit_price_per_product = {
    product_id: round(np.random.uniform(1.15, 100.45), 2)
    for product_id in set(product_ids) 
}

# Génération des quantities
quantities = np.random.randint(1, 101, n_transactions)

# Génération des dates
start_date = pd.to_datetime('2023-01-01')
end_date = pd.to_datetime('2023-01-31')
dates = pd.date_range(start_date, end_date, freq='min').tolist()
datetime = np.random.choice(dates, n_transactions)

# Création de la liste de dictionnaires
transactions = [
    {"id": id, "product_id": product_id, "quantity": quantity, "datetime": dt}
    for id, product_id, quantity, dt in zip(ids, product_ids, quantities, datetime)
]

for t in transactions:
    product_id = t["product_id"]
    t["unit_price"] = unit_price_per_product[product_id]

In [43]:
pd.DataFrame(transactions).head(3)

,id,product_id,quantity,datetime,unit_price
0,1,83,82,2023-01-09 04:27:00,27.22
1,2,63,73,2023-01-21 23:18:00,99.45
2,3,76,45,2023-01-05 04:51:00,29.94


In [5]:
import sqlite3

# On crée l'object connection
conn = sqlite3.connect('transaction.db')
cursor = conn.cursor()

# On crée la table
cursor.execute('''DROP TABLE IF  EXISTS transaction_2023;''')

cursor.execute('''CREATE TABLE IF NOT EXISTS transaction_2023 (
        id INTEGER PRIMARY KEY,
        product_id INTEGER,
        quantity INTEGER,
        unit_price REAL,
        datetime TEXT
    );
''')

# On formatte en liste de tuples + conversion des dates
data_to_insert = [
    (int(t['product_id']), int(t['quantity']), t['unit_price'], t['datetime'].strftime('%Y-%m-%d %H:%M:%S')) 
    for t in transactions
]

# Insérer les données
cursor.executemany('''
    INSERT INTO transaction_2023 (product_id, quantity, unit_price, datetime) VALUES (?, ?, ?, ?)
''', data_to_insert)

# commit et fermeture
conn.commit()
# conn.close()

In [6]:
## data_to_insert

In [7]:
cursor = conn.cursor()
res = cursor.execute("SELECT * FROM transaction_2023;")
res = res.fetchall()
pd.DataFrame(res)

,0,1,2,3,4
0,1,42,37,2.61,2023-01-22 21:43:00
1,2,20,78,56.33,2023-01-22 22:08:00
2,3,112,42,54.65,2023-01-09 14:32:00
3,4,52,73,53.17,2023-01-09 16:41:00
4,5,42,3,2.61,2023-01-24 14:43:00
...,...,...,...,...,...
9995,9996,77,56,65.73,2023-01-19 18:49:00
9996,9997,50,95,13.22,2023-01-16 04:11:00
9997,9998,109,33,82.25,2023-01-23 21:36:00
9998,9999,53,34,69.79,2023-01-19 07:00:00


In [14]:
class Teacher:
    def __init__(self, first_name, teaching_domain):
        self.first_name = first_name
        self.teaching_domain = teaching_domain
        
    def scream(self):
        print("AHHHHHHHHHH")
        
    def teach(self):
        print("Il faut découper en étapes simples les problèmes compliqués")
        
    def greet_student(self, student_name):
        print(f"Bonjour {student_name}")
        for _ in range(3):
            print("Pourrais tu allumer ta caméra ?")
                
    def be_smart_about_it(self, student_name):
        for _ in range(2):
            print(self.greet_student(student_name))
            
        
dumbledore = Teacher("Albus", "life")
rogue = Teacher("Severus", "potions")
mcgonagall = Teacher("Minerva", "metamorphosis")



dumbledore.greet_student("Lucky Potter")


Bonjour Lucky Potter
Pourrais tu allumer ta caméra ?
Pourrais tu allumer ta caméra ?
Pourrais tu allumer ta caméra ?


In [5]:
import sqlite3
from datetime import datetime


class IdDoesNotExist(Exception):
    pass

# Avec un seul curseur
class TransactionManager:
    def __init__(self, db_path):
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS transaction_2023 (
                id INTEGER PRIMARY KEY,
                product_id INTEGER,
                quantity INTEGER,
                unit_price REAL,
                datetime TEXT
            );
        ''')
        self.conn.commit()

    def create(self, records):
        self.cursor.execute('''
            INSERT INTO transaction_2023 (product_id, quantity, unit_price, datetime) 
            VALUES (?, ?, ?, ?)
        ''', (records['product_id'], records['quantity'], records['unit_price'], records['datetime']))
        self.conn.commit()
        return self.cursor.lastrowid

    def read(self, id):
        self.cursor.execute('SELECT * FROM transaction_2023 WHERE id = ?', (id,))
        return self.cursor.fetchone()

    def update(self, id, partial_record):
        updates = ', '.join([f"{k} = ?" for k in partial_record.keys()])
        values = list(partial_record.values()) + [id]
        self.cursor.execute(f'UPDATE transaction_2023 SET {updates} WHERE id = ?', values)
        self.conn.commit()

    def delete(self, id):
        if self.read(id) is None:
            raise IdDoesNotExist(f"id: {id} was not found in table transaction_2023")
            
        self.cursor.execute('DELETE FROM transaction_2023 WHERE id = ?', (id,))
        self.conn.commit()

# Exemple d'utilisation
if __name__ == "__main__":
    # Initialisation
    manager = TransactionManager('transaction.db')

    # Créer une transaction
    new_id = manager.create({
        'product_id': 1,
        'quantity': 10,
        'unit_price': 2.5,
        'datetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    })

    # Lire une transaction
    transaction = manager.read(new_id)
    print(f"Lecture de la transaction: {transaction}")

    # Mettre à jour une transaction
    manager.update(new_id, {'quantity': 20, 'unit_price': 2.75})
    updated_transaction = manager.read(new_id)
    print(f"Transaction mise à jour: {updated_transaction}")

    # Supprimer une transaction
    manager.delete(new_id)
    deleted_transaction = manager.read(new_id)
    print(f"Transaction supprimée, vérification: {deleted_transaction}")

    # Assurez-vous d'utiliser des assertions pour tester que chaque opération fonctionne comme prévu
    assert transaction is not None, "La création a échoué"
    assert updated_transaction[2] == 20 and updated_transaction[3] == 2.75, "La mise à jour a échoué"
    assert deleted_transaction is None, "La suppression a échoué"
    
    try:
        manager.delete(100000000)
    except IdDoesNotExist:
        print("Exception correctly raised")

Lecture de la transaction: (10001, 1, 10, 2.5, '2024-03-01 11:03:06')
Transaction mise à jour: (10001, 1, 20, 2.75, '2024-03-01 11:03:06')
Transaction supprimée, vérification: None
Exception correctly raised


In [ ]:
def update(self, id:int, partial_record:dict):
    record = ", ".join(
        list(map(lambda key: f"'{key}' = {partial_record[key]}", partial_record))
    )
    self.curs.execute(f"UPDATE 'transaction' SET {record} WHERE id = {id}")
    self.conn.commit()

In [1]:
class AllCamerasArentOn(Exception):
    pass

n_students = 19

n_cameras_on = 3

if n_cameras_on < n_students:
    raise AllCamerasArentOn


AllCamerasArentOn: 

In [ ]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///transactions.db')

with engine.connect() as connection:
    result = connection.execute("SELECT * FROM transaction_2023;")
    for row in result:
        print(row)